This script is aiming to find the proper script to make a classicification of the images.

# Import pacakage

In [1]:
import tensorflow as tf
import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

import os
import scipy

# Use in the model
from tensorflow.keras.layers import Dense, Dropout

# Load Images

In [5]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset'
# class_names = os.listdir(data_path)
class_names = ["background", "pharmaceutical", "sharps", "trace_chemo"]

In [6]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.5)

train_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='validation')

Found 4502 images belonging to 4 classes.
Found 4501 images belonging to 4 classes.


In [8]:
# # Show several images
# data, labels = validation_generator.next()

# for i, sample in enumerate(data):
#     plt.imshow(sample)
#     print(class_names[int(labels[i])])
#     plt.show()

# MobileNet Model

In [14]:
gpu_index = 3

In [15]:
# Import mobilenet
# with tf.device('/gpu:' + str(gpu_index)):
mobilenet = tf.keras.applications.MobileNet(include_top=False, input_shape=[224, 224, 3], weights='imagenet')
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [16]:
# Add new output layers.
# with tf.device('/gpu:' + str(gpu_index)):
x = mobilenet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = tf.keras.layers.Dense(4, activation='softmax')(x)

# Combine feature layers with our new output layers.
model = tf.keras.Model(inputs=mobilenet.input, outputs=preds)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

# Train the Model

In [17]:
# defintion of optimizer
# with tf.device('/gpu:' + str(gpu_index)):
# opt = tf.keras.optimizers.SGD(0.001)
opt = tf.train.GradientDescentOptimizer(0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# definition of checkpoint
# with tf.device('/gpu:' + str(gpu_index)):
checkpoint_path = os.path.join(home_path, 'models', 'model_4classes', "model_4_classes.h5")
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True)

In [6]:
#model = tf.keras.models.load_model(os.path.join(home_path + 'models', "model_new_layer.h5"))

In [19]:
# train model
with tf.device('/gpu:3'):
    model.fit_generator(generator=train_generator,
                        validation_data=validation_generator, 
                        epochs=200,
                        callbacks=[cp_callback])

Epoch 1/200
188/188 [==============================] - 143s 760ms/step - loss: 1.4897 - acc: 0.3659 - val_loss: 1.1517 - val_acc: 0.4772
Epoch 2/200
188/188 [==============================] - 135s 716ms/step - loss: 0.9886 - acc: 0.5825 - val_loss: 0.7101 - val_acc: 0.7472
Epoch 3/200
188/188 [==============================] - 135s 717ms/step - loss: 0.6252 - acc: 0.7566 - val_loss: 0.3837 - val_acc: 0.8518
Epoch 4/200
188/188 [==============================] - 135s 718ms/step - loss: 0.4187 - acc: 0.8462 - val_loss: 0.3250 - val_acc: 0.8616
Epoch 5/200
188/188 [==============================] - 136s 721ms/step - loss: 0.2932 - acc: 0.8977 - val_loss: 0.2883 - val_acc: 0.8796
Epoch 6/200
188/188 [==============================] - 136s 724ms/step - loss: 0.1995 - acc: 0.9371 - val_loss: 0.2829 - val_acc: 0.8825
Epoch 7/200
188/188 [==============================] - 136s 721ms/step - loss: 0.1683 - acc: 0.9438 - val_loss: 0.2541 - val_acc: 0.8938
Epoch 8/200
188/188 [====================

Epoch 61/200
188/188 [==============================] - 97s 516ms/step - loss: 0.0073 - acc: 0.9980 - val_loss: 0.1091 - val_acc: 0.9753
Epoch 62/200
188/188 [==============================] - 98s 520ms/step - loss: 0.0100 - acc: 0.9967 - val_loss: 0.1037 - val_acc: 0.9760
Epoch 63/200
188/188 [==============================] - 97s 518ms/step - loss: 0.0106 - acc: 0.9971 - val_loss: 0.1128 - val_acc: 0.9753
Epoch 64/200
188/188 [==============================] - 97s 517ms/step - loss: 0.0112 - acc: 0.9967 - val_loss: 0.1077 - val_acc: 0.9756
Epoch 65/200
188/188 [==============================] - 98s 519ms/step - loss: 0.0088 - acc: 0.9965 - val_loss: 0.1056 - val_acc: 0.9756
Epoch 66/200
188/188 [==============================] - 96s 513ms/step - loss: 0.0101 - acc: 0.9980 - val_loss: 0.0997 - val_acc: 0.9756
Epoch 67/200
188/188 [==============================] - 98s 520ms/step - loss: 0.0082 - acc: 0.9978 - val_loss: 0.0993 - val_acc: 0.9760
Epoch 68/200
188/188 [===================

KeyboardInterrupt: 

In [20]:
tf.contrib.saved_model.save_keras_model(model, home_path + 'models/model_4classes')

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

b'/home/bowen/Documents/Ecomed/models/model_4classes/1551934800'